In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import import_ipynb
import data_preperation as dp
import internal_inconsistencies as ii
import external_inconsistencies as ei


importing Jupyter notebook from data_preperation.ipynb


SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

# 1. Data mismatches in ENTSO-E data

## 1.1 Calculating missing values in data files

In [ ]:
ii.mismatch_raw_data(countries)


# 2. ENTSO-E data representation

## 2.1 Get all the hourly load data of each country to a single dataframe

In [ ]:
load_data,load_dic=dp.load(countries)
display(load_data)


## 2.2 Get all the hourly generation data of each country from each source to a single dataframe

In [ ]:
generation_data,generation_dic=dp.generation(countries)
display(generation_data)

## 2.3 Get all the hourly cross-border transmission data of each country to a single dataframe

In [ ]:
crossborder_data, transmission_data = dp.cross_border(abbr_list)

In [ ]:
display(transmission_data)

### 2.3.1 'cross_border_data' shows the net import/net export of the whole country in hourly timesteps

In [ ]:
display(crossborder_data)

### 2.3.2 Calculate net imports/ net exports of each country in each hour based on generation - load data of the country

In [ ]:
import_export_using_load_gen=dp.import_export_using_load_gen(load_data,generation_data,abbr_list)
display(import_export_using_load_gen)

### 2.3.3 Calculate net imports/ net exports of each country in each hour based on cross-border transmission data of the country

In [ ]:
import_export_using_crossborder_data=dp.import_export_using_crossborder(crossborder_data,abbr_list)
display(import_export_using_crossborder_data)

# 3. Consolidation based on internal data

## 3.1 Obtaining consolidated ENTSO-E data using Internal Sigma Approach

In [ ]:
consolidated_gen_data, consolidated_load_data, consolidated_transmission_data = ii.internal_sigma(load_dic, generation_dic, transmission_data, countries)

## 3.2 Mismatch analysis in the consolidated ENTSO-E data

In [ ]:
ii.mismatch_consolidated_data(consolidated_gen_data, consolidated_load_data,consolidated_transmission_data, countries)

# 4. Comparission of ENTSO-E data with other sources

## 4.1 Comparing ENTSO-E & Eurostat data

### 4.1.1 Comparing annual generation data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_gen_data(generation_data,countries)

### 4.1.2 Comparing annual load data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_load_data(load_data, countries)


### 4.1.3 Comparing annual import/export data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_transmission_data(import_export_using_crossborder_data, countries)